# Tutorial: Quick start
Author info: Jiaqi Cai@Department of Physics, University of Washington, Seattle, WA 98195, email address caidish[at]uw.edu.

This tutorial shows how to use this package to do all the data acqusition from:
- Sweep0D: monitor all the *follow_params* followed parameters as a function of system time. 
- Sweep1D: monitor all the followed parameters when sweeping one paramter. This sweep1D can do sweep one-way or back-and-forth, or continously. 
- Sweep2D: monitor all the followed parameters when sweeping two paramters in a back-and-forth scan. The inner parameter will be swept back and forth and outer parameter will step when one loop finished. 
- SimuSweep: same as sweep1D, but two parameters could be scanned simutaneously. 
- SweepQueue: consist of the queue of sweep0D, sweep1D, sweep2D and SimuSweep and any user defined function. The sweeps will be run one by one.

I will use the MockParabola function from qcodes' test instrument. The simulated device Mockparabola has main parameters *parabola*, *x*,*y*,*z* and *noise*. The output parabola is a quadric function of x,y,z in the presence of noise. 

## Import essential modules

In [1]:
import qcodes as qc
from qcodes.instrument_drivers.mock_instruments import MockParabola

from measureit import (
    GateLeakage,
    SimulSweep,
    Sweep0D,
    Sweep1D,
    Sweep2D,
    get_data_dir,
    get_path,
)
from measureit.tools import ensure_qt, init_database

ensure_qt()
# Set MEASUREIT_FORCE_QT=1 to bypass the probe once Qt is configured.
from measureit.tools.sweep_queue import DatabaseEntry, SweepQueue


MeasureIt data directory: /home/wangs/instrmcptest
To change: measureit.set_data_dir('/path')



In [2]:
# where the data will be saved
print(f"Data directory: {get_data_dir()}")
print(f"Database directory: {get_path('databases')}")

Data directory: /home/wangs/instrmcptest
Database directory: /home/wangs/instrmcptest/Databases


## Define the instrument(s)

We imported a test instrument 'MockParabola' from qcodes' tests. Other supported drivers could be found in [qcodes driver list](https://github.com/QCoDeS/Qcodes/tree/master/qcodes/instrument_drivers) and [contributed driver list](https://github.com/QCoDeS/Qcodes_contrib_drivers/tree/master/qcodes_contrib_drivers/drivers). We also have some user contributed driver (e.g. old lakeshore)

In [3]:
instr0 = MockParabola(name="test_instrument0")
instr0.noise.set(3)
instr0.parabola.label = "Value of instr0"
instr1 = MockParabola(name="test_instrument1")
instr1.noise.set(10)
instr1.parabola.label = "Value of instr1"

What we do is to create two instances of MockParabola. In other cases when people need to deal with multiple instruments, they can be imported and defined one by one. For parameters, the 'label' property will appear as the y-axis name when we dynamically display the measurement result. 

## Sweep 0d to monitor the noise

In [4]:
s = Sweep0D(
    inter_delay=0.1, save_data=True, plot_bin=4, max_time=100
)  # inter_delay is the delay after qcodes collecting all data. inter_delay = 0.05 corresponds to a maximum sampling rate @200S/s.
follow_params = {  # Define which parameters you want to follow (plot and/or save)
    instr0.parabola,
    instr1.parabola,
}
s.follow_param(*follow_params)

The 0d sweep which monitors the output will last for max_time (in unit of seconds), which now is set as 100s. 

In [5]:
try:
    # Make sure database_name and the path are set to the correct values!
    database_name = "Test_database.db"
    exp_name = "testsweep"
    sample_name = "test0d"
    init_database(database_name, exp_name, sample_name, s)
except:
    print("Error opening database")

There is (are) already experiment(s) with the name of testsweep and sample name of test0d in the database.


In [6]:
print(s)

0D Sweep for 100 seconds.


In [7]:
s.start()

In [8]:
ds = qc.dataset.load_by_id(1)

In [9]:
ds.metadata

{'measureit': '{"class": "Sweep0D", "module": "measureit.sweep.sweep0d", "attributes": {"inter_delay": 0.1, "save_data": true, "plot_data": true, "plot_bin": 4, "max_time": 100}, "set_param": null, "follow_params": {"test_instrument1.parabola": ["test_instrument1", "qcodes.instrument_drivers.mock_instruments", "MockParabola"], "test_instrument0.parabola": ["test_instrument0", "qcodes.instrument_drivers.mock_instruments", "MockParabola"]}}'}

Before the max_time reaches, one way to stop is to use ESC on your keyboard. Another way to stop it is run the code below. **Note that closing the plotter's window doesn't neccessarity stops the data acquisition.**

In [10]:
s.is_running

True

In [11]:
s.stop()

When the sweep is running on the background, however, although the data is acquired in another process, the plotting is still requiring CPU time from this notebook's thread. time.sleep() will pause the plotting but preassumely not the data acquisition. 

## Sweep 1d
The sweep 1d function takes one parameter import, which is instr0's x parameter.
It also requires a start point, end point, rate. **If the parameter is not at 'start', it will safely sweep to 'start' with the rate, making sure there is no jump in any physical instrument.**

In [12]:
start = 0
end = 10
rate = 0.02
s = Sweep1D(
    instr0.x,
    start,
    end,
    rate,
    inter_delay=0.05,
    save_data=True,
    bidirectional=True,
    plot_bin=4,
    continual=False,
)
follow_params = {  # Define which parameters you want to follow (plot and/or save)
    instr0.parabola,
    instr1.parabola,
}
s.follow_param(*follow_params)

In [13]:
s._params[1]

<qcodes.parameters.parameter.Parameter: parabola at 140117720935312>

In [14]:
try:
    # Make sure database_name and the path are set to the correct values!
    database_name = "Test_database.db"
    exp_name = "testsweep"
    sample_name = "test1d"
    init_database(database_name, exp_name, sample_name, s)
except:
    print("Error opening database")

There is (are) already experiment(s) with the name of testsweep and sample name of test1d in the database.


In [15]:
print(s)

1D Sweep of x from 0 to 10, with step size 0.02.


In [16]:
ensure_qt()
s.start()

Ramping to our starting setpoint value of 0 (a.u.)
Done ramping x to 0 (a.u.)
Sweeping x to 10 (a.u.)


During the scan, like sweep0d, there are multiple ways to stop it. Besides, the 'spacebar' could revert the scanning axis. 

In [17]:
s.stop()

## Limit test
This limit test will sweep back automatically once a limit is triggered, which can be used for gate limit test. 

In [18]:
gl = GateLeakage(
    instr0.x,
    instr0.parabola,
    max_I=10,
    limit=20,
    step=0.1,
    inter_delay=0.2,
    save_data=False,
)

In [19]:
ensure_qt()
gl.start()

Sweeping x to 0 (a.u.)


In [20]:
gl.stop()

## Simulsweep
The simulsweep function mimics sweep 1D but take two different parameter. The step must be set correctly that the total steps for each parameters are equal. 

In [21]:
parameter_dict_forward = {
    instr0.x: {"start": 0, "stop": 5, "step": 0.02},
    instr1.x: {"start": 0, "stop": 10, "step": 0.04},
}
sweep_args = {
    "bidirectional": True,
    "plot_bin": 4,
    "continual": False,
    "save_data": True,
    "inter_delay": 0.05,
}
s = SimulSweep(parameter_dict_forward, **sweep_args)  # Likely want to save this sweep
follow_params = {  # Define which parameters you want to follow (plot and/or save)
    instr0.parabola,
    instr1.parabola,
}
s.follow_param(*follow_params)

In [22]:
try:
    # Make sure database_name and the path are set to the correct values!
    database_name = "Test_database.db"
    exp_name = "testsweep"
    sample_name = "testSimu"
    init_database(database_name, exp_name, sample_name, s)
except:
    print("Error opening database")

There is (are) already experiment(s) with the name of testsweep and sample name of testSimu in the database.


In [23]:
ensure_qt()
s.start()

Ramping to our starting setpoints.
Already at the values, no ramp needed.
Done ramping!
Starting our sweep.


In [24]:
s.is_running

False

In [25]:
s.stop()

Sweep not currently running. Nothing to stop.


## Sweep Queue
The most important application of this code is that you can freely stack all sweep types and user defined function together. Here I will show a sweepqueue that:
- sweep instr0's x from 0 to 5
- print a dummy string 1
- sweep instr1's x and y from 0 to 5
- print a dummy string 2

In [24]:
def dummystring(index):
    if index == 1:
        print("Lorem ipsum dolor sit amet, consectetur adipiscing elit.")
    elif index == 2:
        print(
            "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."
        )

In [25]:
sq = SweepQueue()
follow_params = {  # Define which parameters you want to follow (plot and/or save)
    instr0.parabola,
    instr1.parabola,
}

SweepQueue accept sweep object and associated database if save_data is set to be True. 

The += operator is overload for SweepQueue, so that the sweep,databaseentry tuple pair can be directly added into it. Same thing as the append function

In [26]:
# set up the sweep
start = 0
end = 5
rate = 0.2
s1 = Sweep1D(
    instr0.x,
    start,
    end,
    rate,
    inter_delay=0.2,
    save_data=True,
    bidirectional=True,
    plot_bin=4,
    continual=False,
)
s1.follow_param(*follow_params)

db_name = "Test_database.db"
db_path = str(get_path("databases") / db_name)
exp_name = "testsweepqueue"
sample_name = "test1d"

# set up the database
db_entry = DatabaseEntry(db_path, exp_name, sample_name)
sq += (db_entry, s1)

In [27]:
# add dummy string print
sq += (dummystring, 1)

In [28]:
# set up the sweep
parameter_dict_forward = {
    instr0.x: {"start": 0, "stop": 5, "step": 0.02},
    instr0.y: {"start": 0, "stop": 5, "step": 0.02},
}
sweep_args = {
    "bidirectional": True,
    "plot_bin": 4,
    "continual": False,
    "save_data": True,
    "inter_delay": 0.1,
}
s2 = SimulSweep(parameter_dict_forward, **sweep_args)  # Likely want to save this sweep
s2.follow_param(*follow_params)
db_name = "Test_database.db"
db_path = str(get_path("databases") / db_name)
exp_name = "testsweepqueue"
sample_name = "testsimulsweep"

# set up the database
db_entry = DatabaseEntry(db_path, exp_name, sample_name)
sq += (db_entry, s2)

In [29]:
# add dummy string print
sq += (dummystring, 2)

The SweepQueue is iterable, so you can iterate it and print every one's name

In [30]:
for n, s in enumerate(sq):
    print(str(n) + ". " + str(s))

0. Database entry saving to /home/wangs/instrmcptest/Databases/Test_database.db with experiment name testsweepqueue and sample name test1d.
1. 1D Sweep of x from 0 to 5, with step size 0.2.
2. functools.partial(<function dummystring at 0x7e356e47b2e0>, 1)
3. Database entry saving to /home/wangs/instrmcptest/Databases/Test_database.db with experiment name testsweepqueue and sample name testsimulsweep.
4. SimulSweep of x from 0 to 5, with step 0.02, and y from 0 to 5, with step 0.02.
5. functools.partial(<function dummystring at 0x7e356e47b2e0>, 2)


In [32]:
ensure_qt()
sq.start()

No sweeps loaded!


In [33]:
sq.is_running()

Sweep queue is not currently running


In [34]:
sq.stop()

No sweep currently running, nothing to stop


## Sweep 2D
The 2D sweep, which is sweeping one parameter back and forth and then step one of the parameter, could be easily achieved by using sweep queue. 
Sweep2d is a good substitute of sweepqueue (though many users still use sweepqueue for 2D sweep) if you only want to take a 2d map in a single file. Here we will sweep Mockparabola's x and y to generate a 2d map. 

In [4]:
# define 2d map
outer_para = instr0.y
outer_dv = 0.5
outer_start = -2.5
outer_end = 2.5

inner_para = instr0.x
inner_dv = 0.1
inner_start = -2.5
inner_end = 2.5

multiplier = 4
inter_delay = 0.1
outer_delay = 1
s = Sweep2D(
    [inner_para, inner_start, inner_end, inner_dv],
    [outer_para, outer_start, outer_end, outer_dv],
    inter_delay=inter_delay,
    outer_delay=outer_delay,
    save_data=True,
    plot_data=True,
    complete_func=None,
    update_func=None,
    plot_bin=5,
    back_multiplier=multiplier,
    out_ministeps=1,
)
follow_params = {  # Define which parameters you want to follow (plot and/or save)
    instr0.parabola,
    instr1.parabola,
}
s.follow_param(*follow_params)
print(s)

Heatmap will follow a random parameter by default
2D Sweep of y from -2.5 to 2.5 with step 0.5, while sweeping x from -2.5 to 2.5 with step 0.1. Heatmap follows Value of instr1. 


In [5]:
# define the parameter to follow by the heatmap
s.follow_heatmap_param([instr0.parabola, instr1.parabola])

In [6]:
print(s)

2D Sweep of y from -2.5 to 2.5 with step 0.5, while sweeping x from -2.5 to 2.5 with step 0.1. Heatmap follows Value of instr0. 


In [7]:
try:
    # Make sure database_name and the path are set to the correct values!
    database_name = "Test_database.db"
    exp_name = "testsweep"
    sample_name = "test2d"
    init_database(database_name, exp_name, sample_name, s)
except:
    print("Error opening database")

There is (are) already experiment(s) with the name of testsweep and sample name of test2d in the database.


In [8]:
ensure_qt()
s.start()

Sweeping y to -2.5 (a.u.)
Ramping y to -2.5 . . . 


In [10]:
s.is_running

False

In [11]:
s.stop()

Sweep not currently running. Nothing to stop.
